In [146]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats import mode

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()

In [164]:
columns = pd.read_csv('./data/VariableDefinitions.csv')

In [44]:
for col, desc in columns[['Variable Definitions', 'Unnamed: 2']].values[2:]:
    print(f'{col:<15} -- {str(desc).strip()}')

user_id         -- nan
REGION          -- the location of each client
TENURE          -- duration in the network
MONTANT         -- top-up amount
FREQUENCE_RECH  -- number of times the customer refilled
REVENUE         -- monthly income of each client
ARPU_SEGMENT    -- income over 90 days / 3
FREQUENCE       -- number of times the client has made an income
DATA_VOLUME     -- number of connections
ON_NET          -- inter expresso call
ORANGE          -- call to orange
TIGO            -- call to Tigo
ZONE1           -- call to zones1
ZONE2           -- call to zones2
MRG             -- a client who is going
REGULARITY      -- number of times the client is active for 90 days
TOP_PACK        -- the most active packs
FREQ_TOP_PACK   -- number of times the client has activated the top pack packages
CHURN           -- variable to predict - Target


In [168]:
#import data
train = pd.read_csv('./data/Train_folds.zip')
test=  pd.read_csv('./data/Test.zip')
submission = pd.read_csv('./data/SampleSubmission.csv')

cat_cols = [
    # 'user_id',
    'REGION',
    # 'TENURE',
    # 'MRG',  # constant
    # 'TOP_PACK',  # wtf column
]

num_cols = [
    'MONTANT',
    'FREQUENCE_RECH',
    'REVENUE',
    'ARPU_SEGMENT',
    'FREQUENCE',
    'DATA_VOLUME',
    'ON_NET', 
    'ORANGE',
    'TIGO',
    'ZONE1',
    'ZONE2',
    'REGULARITY',
    'FREQ_TOP_PACK',
]

target = 'CHURN'

useful_cols = [
    'REGION',
    'TENURE',
    # 'MRG',  # constant
    # 'TOP_PACK',  # wtf column
    'MONTANT',
    'FREQUENCE_RECH',
    'REVENUE',
    'ARPU_SEGMENT',
    'FREQUENCE',
    'DATA_VOLUME',
    'ON_NET', 
    'ORANGE',
    'TIGO',
    'ZONE1',
    'ZONE2',
    'REGULARITY',
    'FREQ_TOP_PACK',
]

mapping = {
    'D 3-6 month': 1,
    'E 6-9 month': 2,
    'F 9-12 month': 3,
    'G 12-15 month': 4,
    'H 15-18 month': 5,
    'I 18-21 month': 6,
    'J 21-24 month': 7,
    'K > 24 month': 8,
}

train['TENURE'] = train['TENURE'].map(mapping) 
test['TENURE'] = test['TENURE'].map(mapping)

train['REGION'] = train['REGION'].fillna('DAKAR')
test['REGION'] = test['REGION'].fillna('DAKAR')

# agg_by_TENURE = pd.read_csv('./data/agg_by_TENURE.csv')

# for nc in tqdm(num_cols):
#     d = agg_by_TENURE[['TENURE', f'{nc}_mean']].to_dict('list')
#     d = dict(zip(d['TENURE'], d[f'{nc}_mean']))
    
#     series = train.progress_apply(lambda x: d[x['TENURE']] if np.isnan(x[nc]) else x[nc], axis=1)
#     train[nc] = series

for nc in tqdm(num_cols):
    train[nc] = train[nc].fillna(train[nc].mean())

  0%|          | 0/13 [00:00<?, ?it/s]

In [169]:
train.sample(20)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN,kfold
1005387,7790712de8531be2a1ef26c36c530e5fa967d61b,DAKAR,8,2000.000000,4.00000,2000.000000,667.000000,5.000000,3366.450167,2.00000,38.000000,0.000000,8.170132,7.553309,NO,24,All-net 500F=2000F;5d,2.000000,0,3
2153225,ffe677c48bf98a5a299903687af3d5a18d375e38,DIOURBEL,8,200.000000,2.00000,171.000000,57.000000,3.000000,3366.450167,277.68914,95.418711,1.000000,8.170132,7.553309,NO,2,NaN,9.272461,0,4
1673940,c6eaa577027deb1f46d331ad8f3ef2f709e047c7,DAKAR,8,500.000000,1.00000,1000.000000,333.000000,3.000000,3366.450167,277.68914,17.000000,23.109253,8.170132,7.553309,NO,13,All-net 500F=2000F;5d,1.000000,0,4
398474,2f60284469af87cd45b1b3f07c12235878d2a0ae,DAKAR,8,1000.000000,1.00000,1000.000000,333.000000,1.000000,704.000000,277.68914,95.418711,23.109253,8.170132,7.553309,NO,3,"Data:1000F=5GB,7d",1.000000,0,0
1614542,bfde0ece0a80b52ed6daa7ace5f3a54f37f31955,SAINT-LOUIS,8,16500.000000,17.00000,15900.000000,5300.000000,20.000000,3366.450167,1213.00000,3.000000,23.000000,8.170132,7.553309,NO,62,On-net 1000F=10MilF;10d,12.000000,0,1
1198821,8e90b92c0bfa9e65a44951c80cd02b2361f67ad7,DAKAR,8,5532.116998,11.52912,5510.810334,1836.942894,13.978141,3366.450167,277.68914,95.418711,23.109253,8.170132,7.553309,NO,2,NaN,9.272461,0,4
2031514,f1748abe145e5054ad7955eb9e2275f08b85c4f4,DAKAR,8,16200.000000,33.00000,15558.000000,5186.000000,35.000000,0.000000,138.00000,379.000000,43.000000,10.000000,7.553309,NO,62,All-net 500F=2000F;5d,13.000000,0,3
400093,2f91bed36f605a05d37a0bf15b01d1af1385a230,KAOLACK,8,2200.000000,8.00000,2188.000000,729.000000,11.000000,78.000000,116.00000,9.000000,10.000000,8.170132,7.553309,NO,26,Mixt 250F=Unlimited_call24H,6.000000,0,1
188950,168689a01c4b7fdd41dbec8135e9cd04d2c20806,DAKAR,8,5500.000000,20.00000,5596.000000,1865.000000,38.000000,3366.450167,16.00000,24.000000,4.000000,8.170132,7.553309,NO,62,NaN,9.272461,0,0
1191494,8db3e89a664786964ee9cc9cd7d40696e15d7483,DAKAR,8,5532.116998,11.52912,5510.810334,1836.942894,13.978141,3366.450167,277.68914,95.418711,23.109253,8.170132,7.553309,NO,13,NaN,9.272461,0,1


In [170]:
final_predictions = []
scores = []

for fold in tqdm(range(5), 'folds'):
    xtrain = train[train['kfold'] == fold][useful_cols]
    ytrain = train[train['kfold'] == fold][target]
    
    xvalid = train[train['kfold'] != fold][useful_cols]
    yvalid = train[train['kfold'] != fold][target]
    
    xtest = test[useful_cols]
    
    scaler = StandardScaler()
    xtrain[num_cols] = scaler.fit_transform(xtrain[num_cols])
    xvalid[num_cols] = scaler.transform(xvalid[num_cols])
    xtest[num_cols] = scaler.transform(xtest[num_cols])
    
    encoder = OrdinalEncoder()
    xtrain[cat_cols] = encoder.fit_transform(xtrain[cat_cols])
    xvalid[cat_cols] = encoder.transform(xvalid[cat_cols])
    xtest[cat_cols] = encoder.transform(xtest[cat_cols])
    
    model = XGBClassifier(
        n_estimators=1000,
        n_jobs=-1,
        random_state=42,
        use_label_encoder=False
    )
    
    model.fit(xtrain, ytrain, eval_metric=roc_auc_score, verbose=1000)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    score = roc_auc_score(yvalid, preds_valid)
    scores.append(score)
    print(fold, score)    

print(np.mean(scores), np.std(scores))

# 0.7696186855892555 0.00043098014634677365 fillna через REGION
# 0.7694106138854746 0.000588962476667447 fillna через TENURE
# 0.7701952731313458 0.0006247026357018513 fillna через mean по колонке

folds:   0%|          | 0/5 [00:00<?, ?it/s]

[08:23:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0 0.7699708651504271
[08:26:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1 0.7702262172644809
[08:28:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2 0.7706396141473278
[08:30:

In [161]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [162]:
submission.CHURN = preds
submission.to_csv("./data/submission-xgb-5-folds-1000-est-42-rs-TENURE-mapped-REGION-fillna-DAKAR-OrdEncoder-StScaler-num-fillna-by-reg.csv", index=False)

In [114]:
cp = train.copy()
# cp['TENURE'] = cp['TENURE'].map(mapping)
# agg = cp[~train['REGION'].isna()].groupby('REGION').agg({
#     'TENURE': lambda x: mode(x)[0],
# })
# agg

In [65]:
agg = cp[~cp['TENURE'].isna()].groupby('REGION').agg(
    {
        'TENURE': [lambda x: mode(x)[0], 'mean', 'median'],
        'MONTANT': ['mean', 'median', 'min', 'max'],
        'FREQUENCE_RECH': ['mean', 'median', 'min', 'max'],
        'REVENUE': ['mean', 'median', 'min', 'max'],
        'ARPU_SEGMENT': ['mean', 'median', 'min', 'max'],
        'FREQUENCE': ['mean', 'median', 'min', 'max'],
        'DATA_VOLUME': ['mean', 'median', 'min', 'max'],
        'ON_NET': ['mean', 'median', 'min', 'max'],
        'ORANGE': ['mean', 'median', 'min', 'max'],
        'TIGO': ['mean', 'median', 'min', 'max'],
        'ZONE1': ['mean', 'median', 'min', 'max'],
        'ZONE2': ['mean', 'median', 'min', 'max'],
        'REGULARITY': ['mean', 'median', 'min', 'max'],
        'FREQ_TOP_PACK': ['mean', 'median', 'min', 'max'],
    })

agg

TENURE                       MONTANT                      \
            <lambda_0>      mean median         mean       median    min   
REGION                                                                     
DAKAR                8  7.849000    8.0  5656.985991  5532.116998   10.0   
DIOURBEL             8  7.893157    8.0  4313.352080  4000.000000   50.0   
FATICK               8  7.918525    8.0  4728.508129  3600.000000   30.0   
KAFFRINE             8  7.924209    8.0  4958.614244  4000.000000   10.0   
KAOLACK              8  7.909090    8.0  5020.838681  4000.000000   25.0   
KEDOUGOU             8  7.903922    8.0  5859.741473  5532.116998  100.0   
KOLDA                8  7.855329    8.0  5346.318787  4000.000000   50.0   
LOUGA                8  7.874229    8.0  4907.906921  4000.000000   50.0   
MATAM                8  7.808342    8.0  5033.363167  4000.000000   50.0   
SAINT-LOUIS          8  7.820046    8.0  5906.227981  4900.000000   50.0   
SEDHIOU              8  7.860532    8.0  4863.582772  4000.000000  100.0   
TAMBACOUNDA          8  7.878200    8.0  5678.677752  4500.000000   40.0   
THIES                8  7.878807    8.0  5805.545417  5000.000000   20.0   
ZIGUINCHOR           8  7.920529    8.0  5440.051233  4700.000000   50.0   

                      FREQUENCE_RECH                 ... ZONE2          \
                  max           mean    median  min  ...   min     max   
REGION                                               ...                 
DAKAR        470000.0      11.593583  11.52912  1.0  ...   0.0  3697.0   
DIOURBEL     159000.0       8.407577   7.00000  1.0  ...   0.0  1346.0   
FATICK       109050.0      10.077854   8.00000  1.0  ...   0.0   143.0   
KAFFRINE     146550.0      10.620642   9.00000  1.0  ...   0.0   219.0   
KAOLACK      125000.0      11.394846  10.00000  1.0  ...   0.0   575.0   
KEDOUGOU     101200.0      10.640237  10.00000  1.0  ...   0.0   652.0   
KOLDA        120700.0      12.172179  10.00000  1.0  ...   0.0   565.0   
LOUGA        197400.0      10.436014   8.00000  1.0  ...   0.0   746.0   
MATAM        122600.0      10.872634   9.00000  1.0  ...   0.0   792.0   
SAINT-LOUIS  165977.0      13.650164  11.00000  1.0  ...   0.0  1167.0   
SEDHIOU       53200.0       9.005234   7.00000  1.0  ...   0.0    56.0   
TAMBACOUNDA  290500.0      12.719694  10.00000  1.0  ...   0.0   214.0   
THIES        148950.0      11.743935  10.00000  1.0  ...   0.0   849.0   
ZIGUINCHOR   132150.0      10.545831   9.00000  1.0  ...   0.0   182.0   

            REGULARITY                FREQ_TOP_PACK                        
                  mean median min max          mean    median  min    max  
REGION                                                                     
DAKAR        22.535461   13.0   1  62      9.368761  9.272461  1.0  713.0  
DIOURBEL     30.444097   29.0   1  62      7.227345  8.000000  1.0  413.0  
FATICK       37.898073   42.0   1  62      8.158413  7.000000  1.0  128.0  
KAFFRINE     38.661352   42.0   1  62      8.654639  8.000000  1.0  258.0  
KAOLACK      37.681531   42.0   1  62      8.892101  8.000000  1.0  210.0  
KEDOUGOU     28.742157   26.0   1  62      9.282270  9.272461  1.0  105.0  
KOLDA        40.116744   45.0   1  62      9.607791  9.000000  1.0  308.0  
LOUGA        38.598195   42.0   1  62      8.351236  7.000000  1.0  203.0  
MATAM        35.050064   36.0   1  62      9.042640  9.000000  1.0  428.0  
SAINT-LOUIS  39.258412   44.0   1  62     10.509105  9.000000  1.0  592.0  
SEDHIOU      30.205835   28.0   1  62      7.770595  7.000000  1.0  106.0  
TAMBACOUNDA  39.650870   44.0   1  62      9.898726  9.000000  1.0  207.0  
THIES        37.920317   42.0   1  62      9.388070  9.000000  1.0  624.0  
ZIGUINCHOR   32.734928   32.0   1  62      9.064624  9.272461  1.0  151.0  

[14 rows x 55 columns]

In [66]:
agg2 = agg.copy()
agg2.columns = ['_'.join(col).strip() for col in agg.columns.values]
agg2

,TENURE_<lambda_0>,TENURE_mean,TENURE_median,MONTANT_mean,MONTANT_median,MONTANT_min,MONTANT_max,FREQUENCE_RECH_mean,FREQUENCE_RECH_median,FREQUENCE_RECH_min,...,ZONE2_min,ZONE2_max,REGULARITY_mean,REGULARITY_median,REGULARITY_min,REGULARITY_max,FREQ_TOP_PACK_mean,FREQ_TOP_PACK_median,FREQ_TOP_PACK_min,FREQ_TOP_PACK_max
REGION,,,,,,,,,,,,,,,,,,,,,
DAKAR,8,7.849000,8.0,5656.985991,5532.116998,10.0,470000.0,11.593583,11.52912,1.0,...,0.0,3697.0,22.535461,13.0,1,62,9.368761,9.272461,1.0,713.0
DIOURBEL,8,7.893157,8.0,4313.352080,4000.000000,50.0,159000.0,8.407577,7.00000,1.0,...,0.0,1346.0,30.444097,29.0,1,62,7.227345,8.000000,1.0,413.0
FATICK,8,7.918525,8.0,4728.508129,3600.000000,30.0,109050.0,10.077854,8.00000,1.0,...,0.0,143.0,37.898073,42.0,1,62,8.158413,7.000000,1.0,128.0
KAFFRINE,8,7.924209,8.0,4958.614244,4000.000000,10.0,146550.0,10.620642,9.00000,1.0,...,0.0,219.0,38.661352,42.0,1,62,8.654639,8.000000,1.0,258.0
KAOLACK,8,7.909090,8.0,5020.838681,4000.000000,25.0,125000.0,11.394846,10.00000,1.0,...,0.0,575.0,37.681531,42.0,1,62,8.892101,8.000000,1.0,210.0
KEDOUGOU,8,7.903922,8.0,5859.741473,5532.116998,100.0,101200.0,10.640237,10.00000,1.0,...,0.0,652.0,28.742157,26.0,1,62,9.282270,9.272461,1.0,105.0
KOLDA,8,7.855329,8.0,5346.318787,4000.000000,50.0,120700.0,12.172179,10.00000,1.0,...,0.0,565.0,40.116744,45.0,1,62,9.607791,9.000000,1.0,308.0
LOUGA,8,7.874229,8.0,4907.906921,4000.000000,50.0,197400.0,10.436014,8.00000,1.0,...,0.0,746.0,38.598195,42.0,1,62,8.351236,7.000000,1.0,203.0
MATAM,8,7.808342,8.0,5033.363167,4000.000000,50.0,122600.0,10.872634,9.00000,1.0,...,0.0,792.0,35.050064,36.0,1,62,9.042640,9.000000,1.0,428.0


In [68]:
agg2.to_csv('./data/agg_by_region.csv')

In [62]:
pd.read_csv('./data/agg_by_tenure.csv')

,TENURE,REGION_<lambda>,MONTANT_mean,MONTANT_median,MONTANT_min,MONTANT_max,FREQUENCE_RECH_mean,FREQUENCE_RECH_median,FREQUENCE_RECH_min,FREQUENCE_RECH_max,...,ZONE2_min,ZONE2_max,REGULARITY_mean,REGULARITY_median,REGULARITY_min,REGULARITY_max,FREQ_TOP_PACK_mean,FREQ_TOP_PACK_median,FREQ_TOP_PACK_min,FREQ_TOP_PACK_max
0,1,DAKAR,3881.252616,3600.000000,100.0,40540.0,7.536481,8.00000,1.0,41.0,...,0.0,187.0,15.885714,12.5,1,62,6.833046,9.272461,1.0,50.0
1,2,DAKAR,6414.517180,5500.000000,100.0,150000.0,11.245900,11.00000,1.0,104.0,...,0.0,1346.0,30.259924,29.0,1,62,8.942172,9.272461,1.0,197.0
2,3,DAKAR,5318.184930,5532.116998,100.0,102300.0,11.563980,11.52912,1.0,85.0,...,0.0,994.0,25.751179,19.0,1,62,9.080666,9.272461,1.0,190.0
3,4,DAKAR,5209.858674,5532.116998,50.0,154500.0,11.321533,11.52912,1.0,101.0,...,0.0,749.0,22.835112,13.0,1,62,8.963391,9.272461,1.0,153.0
4,5,DAKAR,5113.841946,5532.116998,10.0,143100.0,11.216335,11.52912,1.0,99.0,...,0.0,593.0,25.044490,18.0,1,62,8.930489,9.272461,1.0,206.0
5,6,DAKAR,5346.133744,5532.116998,30.0,159000.0,11.662308,11.52912,1.0,101.0,...,0.0,989.0,25.108110,18.0,1,62,9.226331,9.272461,1.0,274.0
6,7,DAKAR,5527.982891,5532.116998,50.0,198000.0,11.976929,11.52912,1.0,92.0,...,0.0,932.0,26.716385,21.0,1,62,9.512385,9.272461,1.0,214.0
7,8,DAKAR,5544.742862,5532.116998,10.0,470000.0,11.530475,11.52912,1.0,133.0,...,0.0,3697.0,28.204974,24.0,1,62,9.280688,9.272461,1.0,713.0


In [79]:
agg3 = cp[(~cp['REGION'].isna() & ~cp['TENURE'].isna())].groupby('REGION').agg({'CHURN': ['mean', 'median']})
agg3.columns = ['_'.join(col).strip() for col in agg3.columns.values]
agg3.to_csv('./data/agg_by_region_churn.csv')

In [85]:
cp['TENURE'] = cp['TENURE'].map(mapping)

agg3 = cp[(~cp['REGION'].isna() & ~cp['TENURE'].isna())].groupby('TENURE').agg({'CHURN': ['mean', 'median']})
agg3.columns = ['_'.join(col).strip() for col in agg3.columns.values]
agg3.to_csv('./data/agg_by_region_tenure.csv')
pd.read_csv('./data/agg_by_region_tenure.csv')

,TENURE,CHURN_mean,CHURN_median
0,1,0.020534,0.0
1,2,0.040109,0.0
2,3,0.034811,0.0
3,4,0.023885,0.0
4,5,0.022966,0.0
5,6,0.024106,0.0
6,7,0.022156,0.0
7,8,0.017714,0.0


In [153]:
agg_by_region = pd.read_csv('./data/agg_by_region.csv')

cp = train.copy()
cp['TENURE'] = cp['TENURE'].map(mapping)
cp['REGION'] = cp['REGION'].fillna('DAKAR')

num_cols = [
    'MONTANT',
    'FREQUENCE_RECH',
    'REVENUE',
    'ARPU_SEGMENT',
    'FREQUENCE',
    'DATA_VOLUME',
    'ON_NET', 
    'ORANGE',
    'TIGO',
    'ZONE1',
    'ZONE2',
    'REGULARITY',
    'FREQ_TOP_PACK',
]

for nc in tqdm(num_cols):
    d = agg_by_region[['REGION', f'{nc}_mean']].to_dict('list')
    d = dict(zip(d['REGION'], d[f'{nc}_mean']))

    series = cp.apply(lambda x: d[x['REGION']] if np.isnan(x[nc]) else x[nc], axis=1)
    cp[nc] = series
cp

  0%|          | 0/13 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [148]:
test[~test['TENURE'].isna()].progress_apply(lambda x: 1)

  0%|          | 0/18 [00:00<?, ?it/s]

user_id           1
REGION            1
TENURE            1
MONTANT           1
FREQUENCE_RECH    1
REVENUE           1
ARPU_SEGMENT      1
FREQUENCE         1
DATA_VOLUME       1
ON_NET            1
ORANGE            1
TIGO              1
ZONE1             1
ZONE2             1
MRG               1
REGULARITY        1
TOP_PACK          1
FREQ_TOP_PACK     1
dtype: int64